In [1]:
import pyodbc
import pandas as pd
import hashlib

In [2]:
import os.path

In [3]:
import time
import datetime

In [4]:
class sql_server:
    def __init__(self):
        self.conn = None
        self.df = None
    def get_connect(self):
        try:
            self.conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                                       'Server=DESKTOP-3GBSUAG;'
                                       'Database=Survey_Sample_A19;'
                                       'Trusted_Connection=yes;')
        except pyodbc.Error as error:
            print(error.args[1])
    def get_cursor(self,query):
        self.cursor = self.conn.execute(query)
        return self.cursor
    def query_execute(self,query):
        self.conn.execute(query)
        self.conn.commit()
    def execute_to_pandas_df(self,query):
        self.df = pd.read_sql(query, self.conn)
    def pandas_df_to_csv(self,path):
        self.df.to_csv(path,index_label=False,index=False,na_rep='NaN')
    def connection_close(self):
        self.conn.close()

In [5]:
file_path = "D:/data_science/dsti_assignment/survey_data_A19/"
survey_struct_query = "SELECT * FROM [dbo].[SurveyStructure];"

In [6]:
def get_strQueryTemplateForAnswerColumn(survey_id,q_id):
    survey_id = str(survey_id)
    q_id = str(q_id)
    query_p1 = "COALESCE((SELECT a.Answer_Value FROM Answer as a WHERE a.UserId = u.UserId AND a.SurveyId = "
    query_p2 = " AND a.QuestionId = "
    query_p3 = "), -1) AS ANS_Q"
    query_all = query_p1+survey_id+query_p2+q_id+query_p3+q_id
    return query_all

In [7]:
def get_strQueryTemplateForNullColumnn(q_id):
    q_id = str(q_id)
    query = "NULL AS ANS_Q"+q_id
    return query

In [8]:
def get_strQueryTemplateOuterUnionQuery(survey_id,dynamic_ques_ans):
    survey_id = str(survey_id)
    dynamic_ques_ans = str(dynamic_ques_ans)
    query_p1 = "SELECT u.UserId,"
    query_p2 = str(survey_id)+" as SurveyId,"
    query_p3 = dynamic_ques_ans + " from [User] as u WHERE EXISTS (SELECT * FROM Answer as a WHERE u.UserId = a.UserId AND a.SurveyId = "+survey_id+")"
    query_all = query_p1+query_p2+query_p3
    return query_all

In [9]:
def currentQuestionCursor_query_fn(currentSurveyID):
    currentSurveyID = str(currentSurveyID)
    query_p1 = "SELECT * FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE SurveyId = "+str(currentSurveyID)
    query_p2 = " UNION SELECT "+currentSurveyID+ " as SurveyId, Q.QuestionId, 0 as InSurvey FROM Question as Q WHERE NOT EXISTS "
    query_p3 = "( SELECT * FROM SurveyStructure as S WHERE S.SurveyId = "+currentSurveyID + " AND S.QuestionId = Q.QuestionId)) as t ORDER BY QuestionId"
    query_all = query_p1+query_p2+query_p3
    return query_all

In [10]:
def fn_GetAllSurveyDataSQL(total_surveys,total_questions):
    strQueryTemplateForAnswerColumn = ''
    strQueryTemplateForNullColumnn = ''
    strQueryTemplateOuterUnionQuery = ''
    currentSurveyId = None

    strCurrentUnionQueryBlock = ''
    strFinalQuery = ''
    
    surveyCursor1 = sql_server()
    surveyCursor1.get_connect()
    
    surveyCursor = surveyCursor1.get_cursor("SELECT SurveyId FROM Survey ORDER BY SurveyId;")
    survey_tracker=0
    for currentSurveyId in surveyCursor:
        question_tracker=1
        currentQuestionCursor_query = currentQuestionCursor_query_fn(currentSurveyId[0])+";"
        currentQuestionCursor1 = sql_server()
        currentQuestionCursor1.get_connect()
        currentQuestionCursor=currentQuestionCursor1.get_cursor(currentQuestionCursor_query)

        currentSurveyIdInQuestion = None
        currentQuestionID = None
        currentInSurvey = None
        strColumnsQueryPart = ''
        question_tracker = 0
        for currentQuestionCursor_row in currentQuestionCursor:
            currentSurveyIdInQuestion = currentQuestionCursor_row[0]
            currentQuestionID = currentQuestionCursor_row[1]
            currentInSurvey = currentQuestionCursor_row[2]
            if currentInSurvey==0:
                strColumnsQueryPart = strColumnsQueryPart+get_strQueryTemplateForNullColumnn(currentQuestionID)
            else:
                strColumnsQueryPart = strColumnsQueryPart+get_strQueryTemplateForAnswerColumn(currentSurveyId[0],currentQuestionID)
            question_tracker = question_tracker+1
            if question_tracker<total_questions:
                strColumnsQueryPart = strColumnsQueryPart+' , '
            else:
                strColumnsQueryPart = strColumnsQueryPart+''
        currentQuestionCursor1.connection_close()

        strCurrentUnionQueryBlock = get_strQueryTemplateOuterUnionQuery(currentSurveyId[0],strColumnsQueryPart)
        strFinalQuery = strFinalQuery + strCurrentUnionQueryBlock
        survey_tracker=survey_tracker+1
        if(survey_tracker<total_surveys):
            strFinalQuery = strFinalQuery+' UNION '
        else:
            strFinalQuery = strFinalQuery+' ;'
    surveyCursor1.connection_close()
    return strFinalQuery

In [11]:
def vw_AllSurveyData_sql(query,update_flag_field):
    if update_flag_field:
        query = "ALTER VIEW [dbo].[vw_AllSurveyData] AS "+query
    else:
        query = "CREATE OR ALTER VIEW [dbo].[vw_AllSurveyData] AS "+query
    return query

In [12]:
def create_survey_all_view(update_flag=0):
    conn2 = sql_server()
    conn2.get_connect()
    survey_count_r = conn2.get_cursor("select count(distinct SurveyId) from [Survey];")
    for row in survey_count_r:
        total_surveys_count = row[0]
    question_count_r = conn2.get_cursor("select count(distinct QuestionId) from [Question];")
    for row in question_count_r:
        total_questions_count = row[0]
    conn2.connection_close()
    get_all_survey_sql_query = fn_GetAllSurveyDataSQL(total_surveys_count,total_questions_count)
    conn = sql_server()
    conn.get_connect()
    conn.execute_to_pandas_df(get_all_survey_sql_query)
    conn.query_execute(vw_AllSurveyData_sql(get_all_survey_sql_query,update_flag))
    
    if update_flag:
        ts = time.time()
        st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        st_str = st.split(' ')[0].replace('-','_')+'_'+st.split(' ')[1]
        old_survey_file = pd.read_csv(file_path+'all_survey_view.csv')
        old_survey_file.to_csv(file_path+'/old_surveys/all_survey_view_'+st_str+'.csv',index_label=False,index=False,na_rep='NaN')
        conn.pandas_df_to_csv(file_path+'all_survey_view.csv')
        print("Updating the file for storing the hash-value of Survey Structure to implement function 'trg_refreshSurveyView' ")
        print("Old all_survey_view.csv is pushed to location :",file_path+"/old_surveys/")
    else:
        conn.pandas_df_to_csv(file_path+'all_survey_view.csv')
        print("1st time all_survey_view.csv is created")
        print("Creating the file for storing the hash-value of Survey Structure to implement function 'trg_refreshSurveyView' ")
    
    conn.execute_to_pandas_df(survey_struct_query)
    survey_structure_hash_val_file = open(file_path+"survey_structure_hash.txt","w+")
    survey_hash_val = hashlib.sha256(conn.df.to_csv().encode()).hexdigest()
    survey_structure_hash_val_file.write(str(survey_hash_val))
    survey_structure_hash_val_file.close()
    conn.connection_close()
        

In [13]:
def compare_survey_structure():
    conn1 = sql_server()
    conn1.get_connect()
    conn1.execute_to_pandas_df(survey_struct_query)
    hash_val_new_survey_struct = hashlib.sha256(conn1.df.to_csv().encode()).hexdigest()
    old_survey_structure_hash_val_file = open(file_path+"survey_structure_hash.txt","r")
    old_survey_structure_hash_val_value = old_survey_structure_hash_val_file.readline()
    conn1.connection_close()
    if hash_val_new_survey_struct!=old_survey_structure_hash_val_value:
        return 1
    else:
        return 2

In [14]:
def view_all_survey():
    print("summery of the Survey data (from the csv file in the system): ")
    all_survey_df = pd.read_csv(file_path+'all_survey_view.csv')
    print("First 5 records")
    print(all_survey_df.head())
    print("Total number of records in all_survey_view.csv:",all_survey_df.shape[0])

In [15]:
def check_survey():
    if os.path.isfile(file_path+'all_survey_view.csv'):
        print("Survey exists")
        print("Checking if updation is required or not")
        if compare_survey_structure()==1:
            print("Survey Structure table got updated !!")
            print("Creating updated view 'vw_AllSurveyData'")
            create_survey_all_view(1)
        else:
            print("No change in Survey Structure table, hence old existing all_survey_view.csv file is summerised below :-")
    else:
        print ("Survey does not exist and hence creating")
        create_survey_all_view()
        print("all_survey_view.csv and vw_AllSurveyData view created in SQL Server and trigger func for survey structure implemented")
    view_all_survey()

In [16]:
def main():
    check_survey()

In [17]:
if __name__ == "__main__":
    main()

Survey exists
Checking if updation is required or not
No change in Survey Structure table, hence old existing all_survey_view.csv file is summerised below :-
summery of the Survey data (from the csv file in the system): 
First 5 records
   UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3  ANS_Q4
0      42         1     3.0    -1.0     NaN     NaN
1     296         1     5.0    -1.0     NaN     NaN
2    1793         1    -1.0     6.0     NaN     NaN
3    2411         1    -1.0     1.0     NaN     NaN
4    2634         1     3.0    -1.0     NaN     NaN
Total number of records in all_survey_view.csv: 4997
